Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

## 1 test

In [ ]:
split(t)

In [7]:
# Converts the input into tuples
# Not the neatest but good enough
def parse_input(s):
    def parse(line):
        aux = line.split('=')
        sensor_x = int(aux[1].split(',')[0])
        sensor_y = int(aux[2].split(':')[0])
        beacon_x = int(aux[3].split(',')[0])
        beacon_y = int(aux[4].split(':')[0])
        return (sensor_x, sensor_y, beacon_x, beacon_y)
    
    return [parse(line) for line in split(s)]

In [ ]:
parsed = parse_input(t)
parsed

For a given sensor/beacon pair, calculate the distance D between them. Anything at a location <= D from the sensor is a not allowed location. For simplicity assume the sensor is at (0,0). So the locations (-|D-y|, y), ..., (|D-y|, y) are the ones we care are not allowed.

In [ ]:
# The row to count number of disallowed positions
y = 10

# A dictionary to keep track of the disallowed ones
d = {}

# Iterate through sensor/beacon pairs to disallow locations
for tup in parsed:
    # Get x1, x2 for the interval of disallowed locations
    sensor_x, sensor_y, beacon_x, beacon_y = tup
    D = abs(sensor_x - beacon_x) + abs(sensor_y - beacon_y)
    offset_y = abs(y - sensor_y)
    offset_x = D - offset_y
    x1, x2 = sensor_x - offset_x, sensor_x + offset_x
    
    for x in range(x1, x2+1):
        d[x] = False

locs = sorted(list(d.keys()))
print(locs)
print(len(locs))

In [10]:
# But now subtract the number of positions that do contain a beacon
for tup in parsed:
    sensor_x, sensor_y, beacon_x, beacon_y = tup
    if beacon_y == y and beacon_x in d:
        del d[beacon_x]
        
locs = sorted(list(d.keys()))
print(locs)
print(len(locs))

[-2, -1, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
26


## 1 run

In [11]:
parsed = parse_input(s)

# The row to count number of disallowed positions
y = 2000000

# A dictionary to keep track of the disallowed ones
d = {}

# Iterate through sensor/beacon pairs to disallow locations
for tup in parsed:
    # Get x1, x2 for the interval of disallowed locations
    sensor_x, sensor_y, beacon_x, beacon_y = tup
    D = abs(sensor_x - beacon_x) + abs(sensor_y - beacon_y)
    offset_y = abs(y - sensor_y)
    offset_x = D - offset_y
    x1, x2 = sensor_x - offset_x, sensor_x + offset_x
    
    for x in range(x1, x2+1):
        d[x] = False

locs = sorted(list(d.keys()))
print(len(locs))

# But now subtract the number of positions that do contain a beacon
for tup in parsed:
    sensor_x, sensor_y, beacon_x, beacon_y = tup
    if beacon_y == y and beacon_x in d:
        del d[beacon_x]
        
locs = sorted(list(d.keys()))
print(len(locs))

5564018
5564017


In [12]:
# For a manual sanity check that the one removed beacon is within these x
print(min(locs), max(locs))

-229697 5334320


## 2 test

I really like Marcos' solution: do my first idea, but efficiently by working on the level of intervals of locations that aren't allowed. In part 1 I used a dictionary to keep track of locations not allowed, but if we're looking for a location that *is* allowed, it's enough to get the intervals that each sensor prohibits and join them together (looking for a gap along the way).

My solution ended up being too clever and would have been a pain if I'd gotten unlucky: you can reason that the allowed location is either on the edge of the 4,000,001 x 4,000,001 square (could check the edge separately), or is on the border of at least two of the sensors' not-allowed regions. So for each pair of sensors, solve the appropriate equations to find those intersection points. I ignored cases where borders overlap (giving a line of solutions) and it turns out that was enough to find the solution.

First idea
- Just run the previous part on each of the 4000000 rows. Slow, but at least I don't have to use tons of memory storing the 4000000^2 locations.
- But it's not running in reasonable time.

Second idea:
- For each of the 4000000 locations, check its distance from the sensors. Print locations if they're outside of the disallowed range of every sensor.
- Also would take forever: over 1 minute for n=10000.

Third idea:
- The previous, but keeping a list of distances to each of the sensors. That way instead of recalculating the distances, we update the list of distances by +-1 depending on how our x-coord compares to those of the sensors.
- Still too slow, takes 20 seconds to iterate through one row of the 4000000 locations.

Fourth idea:
- I can't be iterating through all the locations, but I can leverage that there is *one* free location so it must appear just outside the border of (at least) one of the no-go zones.
- (If I find at least 2 open locations through this method, then it becomes possible that there's a free location that the method can't find. But that won't happen. And anyways, the unfound free locations would be adjacent to previously found locations (might need to iterate this a few times to find all of them)).
- With this method I have to search 75,209,200 locations which is still a lot but much less than before. At 20 seconds per 4,000,000 this would take less than 7 minutes which is not great, but I'm probably on the right track. Oh! Only search positions that are inbounds, should help a lot. Leverage that the intersection of the sensor border is either totally inside the permissible region or intersects at one of the edges
- Improving this a bit, we know the point is either on the edge of the permissible region *or* (not xor) it's on the border of (at least) two sensors' no-go-zones. Assuming the 2nd case, restrict the search to points that satisfy that condition. Those can be found by taking a pair of sensors and solving the equations that define their borders:
$$|x-x_1| + |y-y_1| = d_1 + 1$$
$$|x-x_2| + |y-y_2| = d_2 + 1$$
- Solving for isolated cases is OK: just a bunch of cases with signs. I held off on solving cases where the lines containing the borders of two sensors overlap.

In [8]:
import numpy as np

In [76]:
def further_parse(tup):
    sensor_x, sensor_y, beacon_x, beacon_y = tup
    return (sensor_x, sensor_y, abs(sensor_x - beacon_x) + abs(sensor_y - beacon_y))
parsed = [further_parse(tup) for tup in parse_input(t)]
parsed

[(2, 18, 7),
 (9, 16, 1),
 (13, 2, 3),
 (12, 14, 4),
 (10, 20, 4),
 (14, 17, 5),
 (8, 7, 9),
 (2, 0, 10),
 (0, 11, 3),
 (20, 14, 8),
 (17, 20, 6),
 (16, 7, 5),
 (14, 3, 1),
 (20, 1, 7)]

In [93]:
# Look for points on the border of 2 sensors
locs = {}
lines = []
for i in range(len(parsed)):
    for j in range(i+1, len(parsed)):
        # The 2 sensors
        x1, y1, d1 = parsed[i]
        x2, y2, d2 = parsed[j]
        
        # Lots of sign options!
        for s1 in [-1,1]:
            for s2 in [-1,1]:
                for t1 in [-1,1]:
                    for t2 in [-1,1]:
                        a1 = d1 + 1 + s1 * x1 + t1 * y1
                        a2 = d2 + 1 + s2 * x2 + t2 * y2
                        
                        # Degenerate case: the lines are the same or parallel
                        if t1*s1 == t2*s2:
                            # They overlap
                            if a1*s1 == s2*a2:
                                # We have x + sigma*y = v
                                sigma = t1*s1
                                v = s1*a1
                                
                                # Record in case I need to investigate these
                                lines += [(sigma, v)]
                            # Otherwise they're parallel, no intersect
                        # "Typical" case: they intersect at a single point
                        else:
                            y = (s1*a1 - s2*a2) / (t1*s1 - t2*s2)
                            x = s1*a1 - s1*t1*y
                            
                            # Check we have integers within the bounds
                            if x == int(x) and 0 <= x <= n and 0 <= y <= n:
                                x = int(x)
                                y = int(y)
                                locs[(x,y)] = True

In [94]:
len(locs)

124

In [95]:
for x, y in locs:
    if all([abs(sensor_x - x) + abs(sensor_y - y) > d for sensor_x, sensor_y, d in parsed]):
        print(x,y)

14 11


In [96]:
14*4000000+11

56000011

## 2 run

In [101]:
n = 4000000
parsed = [further_parse(tup) for tup in parse_input(s)]

In [102]:
# Look for points on the border of 2 sensors
locs = {}
lines = []
for i in range(len(parsed)):
    for j in range(i+1, len(parsed)):
        # The 2 sensors
        x1, y1, d1 = parsed[i]
        x2, y2, d2 = parsed[j]
        
        # Lots of sign options!
        for s1 in [-1,1]:
            for s2 in [-1,1]:
                for t1 in [-1,1]:
                    for t2 in [-1,1]:
                        a1 = d1 + 1 + s1 * x1 + t1 * y1
                        a2 = d2 + 1 + s2 * x2 + t2 * y2
                        
                        # Degenerate case: the lines are the same or parallel
                        if t1*s1 == t2*s2:
                            # They overlap
                            if a1*s1 == s2*a2:
                                # We have x + sigma*y = v
                                sigma = t1*s1
                                v = s1*a1
                                
                                # Later: find all integer x,y that lie on that line and within the bounds
                                lines += [(sigma, v)]
                            # Otherwise they're parallel, no intersect
                        # "Typical" case: they intersect at a single point
                        else:
                            y = (s1*a1 - s2*a2) / (t1*s1 - t2*s2)
                            x = s1*a1 - s1*t1*y
                            
                            # Check we have integers within the bounds
                            if x == int(x) and 0 <= x <= n and 0 <= y <= n:
                                x = int(x)
                                y = int(y)
                                locs[(x,y)] = True

In [103]:
len(locs)

840

In [104]:
for x, y in locs:
    if all([abs(sensor_x - x) + abs(sensor_y - y) > d for sensor_x, sensor_y, d in parsed]):
        print(x,y)

2889605 3398893


In [106]:
# Double check before I submit
x, y = 2889605, 3398893
[(abs(sensor_x - x) + abs(sensor_y - y), d) for sensor_x, sensor_y, d in parsed]

[(657464, 657463),
 (928420, 441687),
 (2659955, 899639),
 (657351, 155152),
 (1477595, 261372),
 (793502, 233901),
 (2578937, 1788091),
 (1374646, 721605),
 (3514502, 1996053),
 (1502936, 257380),
 (2675299, 1875794),
 (5391305, 866268),
 (1216384, 449469),
 (730169, 730168),
 (3941446, 1228537),
 (608112, 608111),
 (1294209, 265300),
 (498349, 498348),
 (1559958, 343735),
 (5972127, 1763442),
 (2653003, 1615533),
 (1830283, 436318),
 (1482989, 430283),
 (1028510, 278627)]

In [107]:
4000000*x+y

11558423398893

# Utilities

In [6]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [2]:
t = """
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
...
Sensor at x=20, y=1: closest beacon is at x=15, y=3
"""

In [5]:
s = """
Sensor at x=3291456, y=3143280: closest beacon is at x=3008934, y=2768339
...
Sensor at x=3059723, y=2540501: closest beacon is at x=3008934, y=2768339
"""